In [2]:
import os
import torch

os.chdir('/home/robsyc/Desktop/thesis/MB-VAE-DTI/')
print(os.listdir('.'))
print(os.getcwd())

print(torch.cuda.is_available())
torch.cuda.empty_cache()

['.git', 'utils', 'data', 'bmfm_sm', 'scripts', '.gitignore', 'README.md', 'ESPF', 'notebooks', 'requirements.txt']
/home/robsyc/Desktop/thesis/MB-VAE-DTI
False


/home/robsyc/miniconda3/envs/mbvae_env/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
model

VariationalBranch(
  (blocks): ModuleList(
    (0): VariationalEncoder(
      (block): Sequential(
        (0): Linear(in_features=2048, out_features=512, bias=True)
        (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (2): SiLU()
        (3): Dropout(p=0.1, inplace=False)
        (4): Linear(in_features=512, out_features=512, bias=True)
        (5): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (6): SiLU()
        (7): Dropout(p=0.1, inplace=False)
        (8): Linear(in_features=512, out_features=512, bias=True)
        (9): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (10): SiLU()
        (11): Dropout(p=0.1, inplace=False)
        (12): Linear(in_features=512, out_features=512, bias=True)
        (13): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (14): SiLU()
        (15): Dropout(p=0.1, inplace=False)
        (16): Linear(in_features=512, out_features=512, bias=True)
        (17): LayerNorm((512,), eps=1e-05, 

In [5]:
from utils.modelBuilding import PlainMultiBranch
import torch

# model = VariationalEncoder(
#     input_dim=2048,
#     hidden_dim=512,
#     output_dim=10,
#     depth=8
# )

model = PlainMultiBranch(
    input_dim_list_0=[2048],
    input_dim_list_1=[4170],
    hidden_dim=512,
    latent_dim=1024,
    depth=2
)

x0 = [torch.Tensor(16, 2048)]
x1 = [torch.Tensor(16, 4170)]
out, coeff = model(x0, x1)

# out, loss = model([x, x2])
# out, coeffs = out[0], out[1].mean(0)
print(out.shape)
# print(coeffs)

torch.Size([16])


---
---

# Load Example Data

In [9]:
import h5torch

dataset = h5torch.Dataset(
    "./data/dataset/DAVIS.h5t",
    sampling="coo",
    subset=("unstructured/split_cold", "valid"),
    in_memory=True,
)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=12, shuffle=True)
batch = next(iter(dataloader))
print(batch.keys())

IndexError: index 331 is out of bounds for axis 0 with size 331

In [15]:
print("Molecule drugs")
for key in ["0/Drug_fp", '0/Drug_emb_graph', '0/Drug_emb_image', '0/Drug_emb_text']:
    print("- ", key[2:], batch[key].shape)

print("\nProtein targets")
for key in ['1/Target_fp', '1/Target_emb_ESM', '1/Target_emb_T5', '1/Target_emb_DNA']:
    print("- ", key[2:], batch[key].shape)

print("\nInteraction value: ", batch["central"].shape)

Molecule drugs
-  Drug_fp torch.Size([12, 2048])
-  Drug_emb_graph torch.Size([12, 512])
-  Drug_emb_image torch.Size([12, 512])
-  Drug_emb_text torch.Size([12, 768])

Protein targets
-  Target_fp torch.Size([12, 4170])
-  Target_emb_ESM torch.Size([12, 1280])
-  Target_emb_T5 torch.Size([12, 1024])
-  Target_emb_DNA torch.Size([12, 1024])

Interaction value:  torch.Size([12])


In [6]:
from torch.utils.data import DataLoader, Dataset
import h5torch

def get_dataset(split_type, split_name):
    return h5torch.Dataset(
        "./data/dataset/DAVIS.h5t",
        sampling="coo",
        subset=(f"unstructured/{split_type}", split_name),
        in_memory=True,
    )

class CustomH5Dataset(Dataset):
    def __init__(self, h5_dataset, inputs_0, inputs_1):
        self.dataset = h5_dataset
        self.inputs_0 = inputs_0
        self.inputs_1 = inputs_1

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        batch = self.dataset[idx]
        # Inputs for branch 0 (drug)
        x0 = [batch[key] for key in self.inputs_0]
        # Inputs for branch 1 (target)
        x1 = [batch[key] for key in self.inputs_1]
        # Interaction value
        y = batch['central']
        return x0, x1, y

config = {
        'inputs_0': ['0/Drug_fp'],
        'inputs_1': ['1/Target_fp'],
        'model_type': 'plain',  # 'plain' or 'variational'
}
batch_size = 16

train_dataset = get_dataset("split_rand", 'train')
valid_dataset = get_dataset("split_rand", 'valid')
test_dataset = get_dataset("split_rand", 'test')

# Create DataLoaders
train_loader = DataLoader(
    CustomH5Dataset(train_dataset, config['inputs_0'], config['inputs_1']),
    batch_size=batch_size,
    shuffle=True
)
valid_loader = DataLoader(
    CustomH5Dataset(valid_dataset, config['inputs_0'], config['inputs_1']),
    batch_size=batch_size,
    shuffle=False
)
test_loader = DataLoader(
    CustomH5Dataset(test_dataset, config['inputs_0'], config['inputs_1']),
    batch_size=batch_size,
    shuffle=False
)

In [18]:
for x in sample_batch:
    try:
        print(x.shape)
    except:
        for i in x:
            print(i.shape)

torch.Size([16, 2048])
torch.Size([16, 4170])
torch.Size([16])


In [8]:
import os
import torch
os.chdir('/home/robsyc/Desktop/thesis/MB-VAE-DTI/')
from utils.modelTraining import train_and_evaluate

CONFIGS = {
    'single_view_fp': {
        'inputs_0': ['0/Drug_fp'],
        'inputs_0_shape': [2048],
        'inputs_1': ['1/Target_fp'],
        'inputs_1_shape': [4170],
        'model_type': 'plain',  # 'plain' or 'variational'
    },
    # 'var_single_view_emb': {
    #     'inputs_0': ['0/Drug_emb_graph'],
    #     'inputs_1': ['1/Target_emb_T5'],
    #     'model_type': 'variational',
    # },
}

split_type = 'split_rand'
for exp_name, config in CONFIGS.items():
    print(f"\nRunning experiment: {exp_name}")
    best_valid_loss, test_loss = train_and_evaluate(
        config=config,
        split_type=split_type,
        num_epochs=10,
        batch_size=16
    )
    print(f"Experiment: {exp_name}, Best Valid Loss: {best_valid_loss:.4f}, Test Loss: {test_loss:.4f}")


Running experiment: single_view_fp


IndexError: index 63 is out of bounds for axis 0 with size 63